<a href="https://colab.research.google.com/github/Shresth-Mehta/Image-Classifier/blob/master/RAVI_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the imports

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
from keras import applications
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import BatchNormalization, Convolution2D, MaxPooling2D
from keras.layers.core import Lambda, Dense, Flatten, Dropout
from keras import regularizers 
import keras.utils
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
import os
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
import tarfile
import numpy as np
import PIL
from PIL import Image 
import random
from sklearn.model_selection import train_test_split

# Getting the file from mounted drive

In [0]:
tf = tarfile.open('drive/My Drive/data_set_2.tar')
tf.extractall()
tf.close()

In [0]:
len(os.listdir('data_set_2'))

15

# Preparing the Data Pipeline 

In [0]:
img_size = 128

In [0]:
print(os.listdir("data_set_2"))

['flow charts', 'chemistry lab equipment', 'maps', 'structural formulae', 'physics free body diagrams', 'line ray diagrams', 'diagrams in biology', 'geomtery', 'statistics', 'fields', 'cartoon in comics', 'circuit diagram', 'Tabular data', 'complex graphs', 'venn diagrams']


In [0]:
classes = {}
#x_data = np.zeros([img_size,img_size,3], dtype=np.uint8)
#y_data = np.zeros()
x_data = []
y_data = []

for i, file in enumerate(os.listdir('data_set_2')):
  count = 0
  file_path = os.path.join('data_set_2', file)
  classes[i] = file 
  for j,img in enumerate(os.listdir(file_path)):
    img_path = os.path.join(file_path, img)
    try:
    
      img = Image.open(img_path).convert('RGB')
      img = img.resize((img_size, img_size), PIL.Image.ANTIALIAS)
      x_data.append(np.array(img))
      y_data.append(i)
      count += 1
    except:
      print(img_path)
      os.remove(img_path)
     
  print (classes[i], count)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


flow charts 751
data_set_2/chemistry lab equipment/1126176.png
chemistry lab equipment 557


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (102817600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


maps 779
structural formulae 603
physics free body diagrams 592
line ray diagrams 534
data_set_2/diagrams in biology/adipose-connective-tissue-biology-pinterest-inside-adipose-connective-tissue-labeled-diagram-600x480.gif
diagrams in biology 590
geomtery 549
statistics 654
fields 637
cartoon in comics 686
circuit diagram 603
Tabular data 568
complex graphs 648
venn diagrams 628


In [0]:
print(classes)

{0: 'flow charts', 1: 'chemistry lab equipment', 2: 'maps', 3: 'structural formulae', 4: 'physics free body diagrams', 5: 'line ray diagrams', 6: 'diagrams in biology', 7: 'geomtery', 8: 'statistics', 9: 'fields', 10: 'cartoon in comics', 11: 'circuit diagram', 12: 'Tabular data', 13: 'complex graphs', 14: 'venn diagrams'}


In [0]:
Y_data = keras.utils.to_categorical(y_data, num_classes=15, dtype='float32')

In [0]:
data = list(zip(x_data, Y_data))
random.shuffle(data)
X_data, Y_data = zip(*data)

In [0]:
count = 0
for i,point in enumerate(X_data):
  if point.ndim != 3:
    #print("yes", i)
    count+= 1
    print(Y_data[i])
print(count)

0


In [0]:
X_arr_data = np.stack(X_data,axis=0)

In [0]:
Y_arr_data = np.stack(Y_data, axis = 0)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split( X_arr_data, Y_arr_data, test_size=0.1, random_state=42)

In [0]:
gen = image.ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True, rotation_range=20, width_shift_range=0.08, height_shift_range=0.08, zoom_range=0.08, horizontal_flip = True)
gen.fit(X_train)
batches = gen.flow(X_train, Y_train, batch_size=32)
val_batches = gen.flow(X_test, Y_test, batch_size=32)

# Model Class

In [0]:
def vgg_4():
  x = applications.vgg16.VGG16(weights = 'imagenet', include_top = False, input_shape = (img_size,img_size, 3))
  X = x.output
  X = GlobalAveragePooling2D()(X)
  X = BatchNormalization()(X)
  X = Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))(X)
  X = Dropout(0.2)(X)
  X = Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))(X)
  X = Dropout(0.25)(X)
  X = BatchNormalization()(X)
  X = Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))(X)
  X = Dropout(0.20)(X)
  X = Dense(120, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))(X)
  X = BatchNormalization()(X)
  pred = Dense(15, activation = 'softmax')(X)
  model = Model(inputs = x.input, output = pred)
  
  return model

In [0]:
model_6 = vgg_4()

W0802 15:33:28.962539 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 15:33:29.032088 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 15:33:29.052899 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 15:33:29.100653 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 1s 0us/step


W0802 15:33:30.499074 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0802 15:33:30.500061 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0802 15:33:34.057953 140610465793920 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(i

In [0]:
for layer in model_6.layers[:15]:
  layer.trainable = False
for layer in model_6.layers[15:]:
  layer.trainable = True

In [0]:
model_6.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0802 15:33:34.677537 140610465793920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
history_6 = model_6.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                    validation_data=val_batches, validation_steps=val_batches.n)

W0802 15:33:34.860817 140610465793920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
8441/8441 [==============================] - 1161s 138ms/step - loss: 1.4957 - acc: 0.8088 - val_loss: 1.3301 - val_acc: 0.7471


In [0]:
print(history_6.history)

{'val_loss': [1.330149906448623], 'val_acc': [0.7470512033858052], 'loss': [1.495894228815138], 'acc': [0.8088590007195218]}


In [0]:
#model_6.save('drive/My Drive/model_6.h5')

In [0]:
print(os.listdir('drive/My Drive/ncert_images'))

['femh103', 'femh101', 'femh102', 'femh105', 'femh104', 'femh106', 'femh108', 'femh107', 'femh109', 'femh111', 'femh110', 'femh112', 'femh113', 'fesc102', 'fesc101', 'femh114', 'fesc103', 'fesc104', 'fesc105', 'fesc106', 'fesc109', 'fesc110', 'fesc111', 'fesc107', 'fesc108', 'fesc116', 'fesc113', 'fesc112', 'fesc115', 'fesc114', 'fess107', 'fess102', 'fess106', 'fess103', 'fess104', 'fess101', 'fess105', 'fess109', 'fess108', 'fess112', 'fess111', 'fess110', 'fess202', 'fess203', 'fess204', 'fess206', 'fess201', 'fess205', 'fess207', 'fess208', 'fess301', 'fess302', 'fess303', 'fess304', 'fess305', 'fess307', 'fess309', 'fess306', 'fess308', 'gemh101', 'gemh102', 'gemh104', 'gemh103', 'gemh106', 'gemh105', 'gemh108', 'gemh107', 'gemh109', 'gemh110', 'gemh111', 'gemh112', 'gemh113', 'gemh115', 'gemh114', 'gesc101', 'gesc103', 'gesc102', 'gesc105', 'gesc104', 'gesc106', 'gesc108', 'gesc107', 'gesc110', 'gesc111', 'gesc109', 'gesc113', 'gesc112', 'gesc114', 'gesc115', 'gesc116', 'gesc118'

In [0]:
os.mkdir('drive/My Drive/45_k_data_set')

In [0]:
for folder_name in classes.values():
  print(folder_name)

flow charts
chemistry lab equipment
maps
structural formulae
physics free body diagrams
line ray diagrams
diagrams in biology
geomtery
statistics
fields
cartoon in comics
circuit diagram
Tabular data
complex graphs
venn diagrams


In [0]:
for folder_name in classes.values():
  folder_path = os.path.join('drive/My Drive/45_k_data_set',folder_name)
  os.mkdir(folder_path)

## **Suggested Method:** *** Labelling new images using the trained classifier to increase the dataset size***

In [0]:
from tqdm import tqdm_notebook as tqdm

In [0]:
for folder in tqdm(os.listdir('drive/My Drive/ncert_images')):
  folder_path = os.path.join('drive/My Drive/ncert_images', folder)
  for img in os.listdir(folder_path):
    x_img = []
    img_path = os.path.join(folder_path, img)
    try:  
      img_open = Image.open(img_path).convert('RGB')
      img_1 = img_open.resize((img_size, img_size), PIL.Image.ANTIALIAS)
      x_img.append(np.array(img_1))
      x_arr_data = np.stack(x_img,axis = 0)
      y = model_6.predict(x_arr_data)
      y = np.argmax(y)
      save_folder_path = os.path.join('drive/My Drive/45_k_data_set',classes[y])
      save_path = os.path.join(save_folder_path,img)
      img_open.save(save_path,'JPEG')
    except:
      print(img_path)

In [0]:
os.mkdir('drive/My Drive/45_k_data_set/null')

In [0]:
count = 0
for folder in tqdm(os.listdir('drive/My Drive/ncert_images')):
  folder_path = os.path.join('drive/My Drive/ncert_images', folder)
  
  for img in os.listdir(folder_path):
    x_img = []
    img_path = os.path.join(folder_path, img)  
    img_open = Image.open(img_path).convert('RGB')
    img_1 = img_open.resize((img_size, img_size), PIL.Image.ANTIALIAS)
    x_img.append(np.array(img_1))
    x_arr_data = np.stack(x_img,axis = 0)
    y = model_6.predict(x_arr_data)
    count += 1
    z = np.amax(y)
    y = np.argmax(y)
    if(z > 0.8):
          
      save_folder_path = os.path.join('drive/My Drive/45_k_data_set',classes[y])
      img_name = folder+"_"+img
      save_path = os.path.join(save_folder_path,img_name)
      img_open.save(save_path)
    elif(z <= 0.8):
      save_folder_path = 'drive/My Drive/45_k_data_set/null'
      img_name = "_null"+img
      save_path = os.path.join(save_folder_path,img_name)
      img_open.save(save_path)
          
  print(folder,count)   

# Removing corrupted images

In [0]:
for i in remove_list[:]:
  os.remove(i)
  print("removed: ",i)

removed:  drive/My Drive/ncert_images/lemh111/Image0001.svg
removed:  drive/My Drive/ncert_images/lemh205/Image0001.svg
